In [2]:
import os


EPOCHS = 100
BATCH_SIZE = 16
ARCH = "flor"

IMG_SIZE = (1024, 128, 1)
DATA_ROOT_PATH = "..\\data"
IMAGES_PATH = os.path.join(DATA_ROOT_PATH, "images", "data")
IMAGES_PATH

'..\\data\\images\\data'

In [3]:
vocab = None

with open(os.path.join(DATA_ROOT_PATH, "hindi_vocab.txt"),encoding="utf-8") as f:
  vocab = f.readlines()

idx_to_vocab = {i:value.strip() for i, value in enumerate(vocab)}
vocab_to_idx = {value:key for key, value in idx_to_vocab.items()}

In [4]:
train_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_train.txt"), encoding="utf-8") as f:
  train_data = f.readlines()

In [5]:
valid_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_val.txt"), encoding="utf-8") as f:
  valid_data = f.readlines()

In [6]:
test_data = None

with open(os.path.join(DATA_ROOT_PATH, "new_val.txt"), encoding="utf-8") as f:
  test_data = f.readlines()

In [7]:
charl = None

with open( "charList.txt", encoding="utf-8") as f:
  charl = f.readlines()

In [8]:
charl=charl[0]

In [9]:
len(charl)

109

In [10]:
len(vocab)

11031

In [28]:
m=0
for i in range(len(vocab)):
    s=idx_to_vocab[i]
    m=max(m,len(s))
m

27

In [30]:
len(train_data)

69803

In [12]:
len(d)

69802

In [25]:
d=[]
path_to_label={}
for i in range(len(train_data)):
    label=train_data[i].split()
    curr_img_path = label[0]
    curr_img_path = "\\".join(curr_img_path.split("/")[2:])
    curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)
    if os.path.exists(curr_img_path):
        d.append(curr_img_path)
        path_to_label[curr_img_path]=int(label[1])

# train data labels

In [13]:
train_data_labels=[]
for i in range(len(train_data)):
    z=[]
    for j in range(27):
        z.append(0)
    
    label=train_data[i].split()
    curr_img_path = label[0]
    curr_img_path = "\\".join(curr_img_path.split("/")[2:])
    curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)
    if os.path.exists(curr_img_path):
        label=idx_to_vocab[int(label[1])]
        for k in range(len(label)):
            for r in range(len(charl)):
                if label[k]==charl[r]:
                    z[k]=r
        train_data_labels.append(z)

In [14]:
len(train_data_labels)

69802

In [42]:
train_data_labels[0]

[57,
 60,
 3,
 23,
 21,
 60,
 3,
 23,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# train data generation

In [ ]:
curr_img_path = curr_data[0]
curr_label = vocab_to_idx[curr_data[1]]

curr_img_path = "\\".join(curr_img_path.split("/")[2:])
curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)

In [50]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as im
from scipy.ndimage import interpolation as inter

def find_score(arr, angle):
    data = inter.rotate(arr, angle, reshape=False, order=0)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score
def skew_c(img):
    #img = im.open(input_file)
    # convert to binary
    wd, ht = img.size
    pix = np.array(img.convert('1').getdata(), np.uint8)
    bin_img = 1 - (pix.reshape((ht, wd)) / 255.0)
    plt.imshow(bin_img, cmap='gray')
    #plt.savefig('binary.png')
    delta = 1
    limit = 5
    angles = np.arange(-limit, limit+delta, delta)
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)
    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    #print('Best angle: {}'.formate(best_angle))
    # correct skew
    data = inter.rotate(bin_img, best_angle, reshape=False, order=0)
    img = im.fromarray((255 * data).astype("uint8")).convert("RGB")
    return img

In [16]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from PIL import Image, ImageOps
from data import preproc as pp

In [ ]:
t_data=[]
for i in range(len(train_data)):
    curr_data=train_data[i].split()
    curr_img_path = curr_data[0]
    curr_img_path = "\\".join(curr_img_path.split("/")[2:])
    curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)
    if os.path.exists(curr_img_path):
        curr_img=skew_c(curr_img_path)
        curr_img = ImageOps.grayscale(curr_img)
        curr_img = pp.preprocess(curr_img_path,IMG_SIZE )
        t_data.append(curr_img)

C:\Users\pavan\AppData\Local\Temp\ipykernel_16052\567187823.py:8: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  data = inter.rotate(arr, angle, reshape=False, order=0)
C:\Users\pavan\AppData\Local\Temp\ipykernel_16052\567187823.py:31: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  data = inter.rotate(bin_img, best_angle, reshape=False, order=0)


In [34]:
##label generator
def label_g(l):
    label=idx_to_vocab[l]
    z=[]
    for j in range(27):
        z.append(0)
    for k in range(len(label)):
        for r in range(len(charl)):
            if label[k]==charl[r]:
                z[k]=r
    return z   

In [53]:
def preprocess1(curr_img_path):
    img=tf.io.read_file(curr_img_path)
    path=os.path.abspath(img)
    print(path)
    curr_img=skew_c(img)
    curr_img = ImageOps.grayscale(curr_img)
    curr_img = pp.preprocess(curr_img_path,IMG_SIZE )
    plt.imshow(curr_img, cmap='gray')
    
    #label=label_g(path_to_label[curr_img_path])
    return curr_img

In [40]:
tf_dataset=tf.data.Dataset.list_files(d)

In [54]:
tf_dataset.map(preprocess1)

TypeError: in user code:

    File "C:\Users\pavan\AppData\Local\Temp\ipykernel_6892\1139157822.py", line 3, in preprocess1  *
        path=os.path.abspath(img)
    File "C:\Users\pavan\anaconda3\envs\pavan\lib\ntpath.py", line 566, in abspath  *
        return normpath(_getfullpathname(path))

    TypeError: _getfullpathname: path should be string, bytes or os.PathLike, not Tensor


In [32]:
for i in tf_dataset.take(2):
    c=i.numpy()
    c=c.decode("utf-8")
    print(path_to_label[c])

4156
2710


In [20]:
len(tf_dataset)

69802

# model


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.constraints import MaxNorm

from network.layers import FullGatedConv2D, GatedConv2D, OctConv2D
from tensorflow.keras.layers import Conv2D, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.layers import Input, Add, Activation, Lambda, MaxPooling2D, Reshape

In [ ]:
pip install tensorflow


In [9]:

def flor(input_size, d_model):
    """
    Gated Convolucional Recurrent Neural Network by Flor et al.
    """

    input_data = Input(name="input", shape=input_size)

    cnn = Conv2D(filters=16, kernel_size=(3, 3), strides=(2, 2), padding="same", kernel_initializer="he_uniform")(input_data)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)
    cnn = FullGatedConv2D(filters=16, kernel_size=(3, 3), padding="same")(cnn)

    cnn = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform")(cnn)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)
    cnn = FullGatedConv2D(filters=32, kernel_size=(3, 3), padding="same")(cnn)

    cnn = Conv2D(filters=40, kernel_size=(2, 4), strides=(2, 4), padding="same", kernel_initializer="he_uniform")(cnn)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)
    cnn = FullGatedConv2D(filters=40, kernel_size=(3, 3), padding="same", kernel_constraint=MaxNorm(4, [0, 1, 2]))(cnn)
    cnn = Dropout(rate=0.2)(cnn)

    cnn = Conv2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform")(cnn)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)
    cnn = FullGatedConv2D(filters=48, kernel_size=(3, 3), padding="same", kernel_constraint=MaxNorm(4, [0, 1, 2]))(cnn)
    cnn = Dropout(rate=0.2)(cnn)

    cnn = Conv2D(filters=56, kernel_size=(2, 4), strides=(2, 4), padding="same", kernel_initializer="he_uniform")(cnn)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)
    cnn = FullGatedConv2D(filters=56, kernel_size=(3, 3), padding="same", kernel_constraint=MaxNorm(4, [0, 1, 2]))(cnn)
    cnn = Dropout(rate=0.2)(cnn)

    cnn = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_initializer="he_uniform")(cnn)
    cnn = PReLU(shared_axes=[1, 2])(cnn)
    cnn = BatchNormalization(renorm=True)(cnn)

    cnn = MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding="valid")(cnn)

    shape = cnn.get_shape()
    bgru = Reshape((shape[1], shape[2] * shape[3]))(cnn)

    bgru = Bidirectional(GRU(units=128, return_sequences=True, dropout=0.5))(bgru)
    bgru = Dense(units=256)(bgru)

    bgru = Bidirectional(GRU(units=128, return_sequences=True, dropout=0.5))(bgru)
    output_data = Dense(units=d_model+1, activation="softmax")(bgru)

    return (input_data, output_data)

In [10]:
model=flor(IMG_SIZE,110)

In [ ]:
ctc_l=tf.nn.ctc_loss(
    tf.zeros((16,50),),
    sequence_length,
    preprocess_collapse_repeated=False,
    ctc_merge_repeated=True,
    ignore_longer_outputs_than_inputs=False,
    time_major=True
)

In [15]:
lossc=tf.nn.ctc_loss()

TypeError: Missing required positional argument